In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import bernoulli, binom
import math
import random


In [11]:
df1 = pd.read_csv('data_policy1.csv', header = None).rename(columns = {0: 'st', 1: 'at', 2: 'rt', 3: 'st1'})
df2 = pd.read_csv('data_policy2.csv', header = None).rename(columns = {0: 'st', 1: 'at', 2: 'rt', 3: 'st1'})

In [169]:
def modelbased(data, alpha = 0, T = 10**6):
    gamma = 0.95
    S = 4
    A = 2
    data = data[0:T]

    N_ss = np.ones((A, S, S))
    for a in range(A):
        for s in range(S):
            for s_m in range(S):
                N_ss[a][s][s_m] += len(data[(data['at'] == a) & (data['st'] == s) & (data['st1'] == s_m)])
    P_ss = np.array([[[((N_ss[a][s][s_m] + alpha) / (sum(N_ss[a][s]) + (alpha * S))) for s_m in range(S)] for s in range(S)] for a in range(A)])

    r_s = np.ones((A, S))
    for a in range(A):
        for s in range(S):
            r_s[a][s] = (sum(data[(data['st'] == s) & (data['at'] == a)]['rt']) + alpha) / (sum(N_ss[a][s]) + alpha)

    Q_t = np.ones((A, S))
    for a in range(A):
        Q_t[a] = np.linalg.inv(np.identity(S) - gamma * P_ss[a]) @ np.array(r_s[a])

    return Q_t

def QL(data, alpha0):
    gamma = 0.95
    S = 4
    A = 2
    Q_t_arr = np.zeros((A, S))
    Q_t = np.zeros((A, S))
    for t in range(len(data)):
        s = data['st'][t]
        if alpha0 == "alpha_a":
            alpha = 10 / ((t+1)**(2/3) + 1)
        if alpha0 == "alpha_b":
            N_t = sum((data[:t+1]['st'] == s) & (data[:t+1]['at'] == data['at'][t]))
            alpha = 10 / (N_t**(2/3) + 1)
            
        #get V_{t+1}
        a = data['at'][t]
        Q_t[a][s] += alpha * (data['rt'][t] + (gamma * (Q_t[a][data['st1'][t]])) - Q_t[a][s])
        Q_t_arr = np.vstack([Q_t_arr, Q_t])

    return Q_t

def make_data(P, r, n = 10**6):
    S = len(r[0])

    s = 0
    a = random.randint(0, 1)
    r1 = r[a][s]
    s1 = random.choices([0, 1, 2, 3], P[a][s])[0]

    dataset = {'st': [s],
               'at': [a],
               'rt': [r1],
               'st1': [s1]}
    
    for t in range(n):
        s = s1
        a = random.randint(0, 1)
        r1 = r[a][s]
        s1 = random.choices([0, 1, 2, 3], P[a][s])[0]
        row = [s, a, r1, s1]
        for key in dataset:
            dataset[key].append(row[list(dataset.keys()).index(key)])
        
    return dataset

In [141]:
#P_ss[a][s][s']
P_ss = np.array([[[0.95, 0.05, 0, 0], 
                  [0.95, 0, 0.05, 0], 
                  [0, 0.95, 0, 0.05], 
                  [0, 0, 0.95, 0.05]],
                 [[0.6, 0.4, 0, 0],
                  [0.05, 0.55, 0.4, 0],
                  [0, 0.05, 0.55, 0.4],
                  [0, 0, 0.4, 0.6]]])

#r_ss[a][s]
r_s = np.array([[0.05, 0, 0, 0], 
               [0, 0, 0, 1]])

In [165]:
gen_df = pd.DataFrame(make_data(P_ss, r_s, n = 10000))

In [167]:
data = gen_df
print(modelbased(data))
print(QL(data, alpha0 = "alpha_a"))
print(QL(data, alpha0 = "alpha_b"))

[[0.95234404 0.90053188 0.85076373 0.80641667]
 [5.93850907 6.70624979 7.71070039 8.94261201]]
[[0.94841069 0.90057814 0.85283629 0.80983783]
 [2.86948416 3.35752028 3.97067896 4.88458194]]
[[0.94146501 0.89312245 0.84385125 0.8002222 ]
 [5.66973447 6.58400408 7.7379773  8.93823399]]


In [137]:
model_df_dist = [np.linalg.norm(V_star1 - modelbased(df1, T = t), ord=np.inf) for t in range(25000)]
TD_df1_alphaa_dist = [np.linalg.norm(V_star1 - V_pi, ord=np.inf) for V_pi in TD_df1_alphaa_arr]
TD_df1_alphab_dist = [np.linalg.norm(V_star1 - V_pi, ord=np.inf) for V_pi in TD_df1_alphab_arr]

model_df2_dist = [np.linalg.norm(V_star2 - modelbased(df2, T = t), ord=np.inf) for t in range(25000)]
TD_df2_alphaa_dist = [np.linalg.norm(V_star2 - V_pi, ord=np.inf) for V_pi in TD_df2_alphaa_arr]
TD_df2_alphab_dist = [np.linalg.norm(V_star2 - V_pi, ord=np.inf) for V_pi in TD_df2_alphab_arr]

1

In [154]:
r_s[0][0]

0.05

In [168]:
10**6

1000000